# 📓 Препроцессинг графа аккаунтов и транзакций
Ниже ‒ **полностью локальный** pipeline в стиле Jupyter Notebook, который:

1. Загружает `accounts.csv` и `transactions.csv`
2. Строит ориентированный граф `NetworkX`
3. Считает структурные и финансовые признаки узлов
4. Конвертирует граф в объект `torch_geometric.data.Data`
5. Делает маски `train / val / test`
6. (Опционально) сериализует результат на диск

Работает целиком без TigerGraph.


## 📦 Установка зависимостей

In [ ]:
!pip install torch==2.2.2 torch_geometric==2.5.3 networkx pandas numpy -q

## 🚀 Импорты

In [ ]:
from pathlib import Path
import networkx as nx
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

print(torch.__version__)

## 📂 Пути к данным

In [ ]:
ACCOUNTS_CSV = Path('accounts.csv')      # обновите при необходимости
TXS_CSV      = Path('transactions.csv')   # обновите при необходимости

assert ACCOUNTS_CSV.exists(), f'{ACCOUNTS_CSV} not found'
assert TXS_CSV.exists(), f'{TXS_CSV} not found'

## 📥 Загрузка CSV

In [ ]:
acc_df = pd.read_csv(ACCOUNTS_CSV, header=None, names=['id', 'label'])
tx_df  = pd.read_csv(TXS_CSV, header=None, names=['src', 'dst', 'amount', 'timestamp'])

display(acc_df.head())
display(tx_df.head())

## 🏗️ Сборка графа

In [ ]:
G = nx.DiGraph()

# add nodes
for _, row in acc_df.iterrows():
    G.add_node(row.id, label=int(bool(row.label)))

# add edges
for _, row in tx_df.iterrows():
    G.add_edge(row.src, row.dst, amount=float(row.amount), ts=float(row.timestamp))

print(f'Граф содержит {G.number_of_nodes():,} узлов и {G.number_of_edges():,} рёбер')

## 🧮 Фичи узлов

In [ ]:
# Структурные признаки
in_deg  = dict(G.in_degree())
out_deg = dict(G.out_degree())

# Финансовые суммы
sent_sum = {n: 0.0 for n in G.nodes()}
recv_sum = {n: 0.0 for n in G.nodes()}
for u, v, d in G.edges(data=True):
    amt = d['amount']
    sent_sum[u] += amt
    recv_sum[v] += amt

# PageRank и кластерный коэффициент
pr = nx.pagerank(G, alpha=0.85)
clust = nx.clustering(G.to_undirected())

# Записываем фичи в граф
for n in G.nodes():
    G.nodes[n].update({
        'in_deg':      in_deg.get(n, 0),
        'out_deg':     out_deg.get(n, 0),
        'sent_sum':    sent_sum[n],
        'recv_sum':    recv_sum[n],
        'net_sum':     sent_sum[n] - recv_sum[n],
        'pagerank':    pr[n],
        'clustering':  clust[n],
    })

print('Пример фич узла:', list(G.nodes(data=True))[0])

## 🔄 Конвертация в `torch_geometric.data.Data`

In [ ]:
num_attr_keys = ['in_deg','out_deg','sent_sum','recv_sum','net_sum','pagerank','clustering']

for n in G.nodes():
    G.nodes[n]['x'] = torch.tensor([float(G.nodes[n][k]) for k in num_attr_keys], dtype=torch.float)

data = from_networkx(G, group_node_attrs=['x'])
data.y = torch.tensor([G.nodes[n].get('label', -1) for n in G.nodes()], dtype=torch.long)

print(data)

## ✂️ Train / Val / Test сплит

In [ ]:
torch.manual_seed(42)
N = data.num_nodes
perm = torch.randperm(N)
n_train = int(0.6 * N)
n_val   = int(0.2 * N)

data.train_mask = torch.zeros(N, dtype=torch.bool)
data.val_mask   = torch.zeros(N, dtype=torch.bool)
data.test_mask  = torch.zeros(N, dtype=torch.bool)

data.train_mask[perm[:n_train]]           = True
data.val_mask[perm[n_train:n_train+n_val]] = True
data.test_mask[perm[n_train+n_val:]]      = True

print(f'Train: {data.train_mask.sum().item()}, Val: {data.val_mask.sum().item()}, Test: {data.test_mask.sum().item()}')

## 💾 Сохранение PyG-объекта

In [ ]:
OUT_PATH = Path('eth_graph.pt')
torch.save(data, OUT_PATH)
print(f'Сериализовано в {OUT_PATH.resolve()}')

### 🎉 Готово! Далее можно переходить к построению и обучению GNN‑модели.

## 🧠 Определение GNN‑модели (GCN)

In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, in_channels: int, hidden_channels: int, num_classes: int):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

print('GCN model ready')

## 🏃‍♂️ Обучение и валидация

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(data.num_node_features, 64, int(data.y.max().item()) + 1).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def accuracy(mask):
    model.eval()
    logits = model(data.x, data.edge_index)
    preds = logits.argmax(dim=1)
    correct = (preds[mask] == data.y[mask]).sum().item()
    return correct / int(mask.sum())

for epoch in range(1, 201):
    loss = train()
    if epoch % 10 == 0:
        train_acc = accuracy(data.train_mask)
        val_acc = accuracy(data.val_mask)
        print(f'Epoch {epoch:03d} | Loss {loss:.4f} | Train Acc {train_acc:.3f} | Val Acc {val_acc:.3f}')

test_acc = accuracy(data.test_mask)
print(f'✅ Test Accuracy: {test_acc:.3f}')

## 💾 Сохранение обученной модели

In [ ]:
MODEL_PATH = Path('gcn_model.pt')
torch.save(model.state_dict(), MODEL_PATH)
print(f'Модель сохранена в {MODEL_PATH.resolve()}')